# 데이터셋 구축

데이터를 정제하고, YOLO 데이터셋 구조로 재구성합니다.

**YOLO 데이터셋 구조**

```
root/
├── train/
│   ├── images/
│   └── labels/
└── val/
    ├── images/
    └── labels/
```

혹은


```
root/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```


## 01. 파일 업로드

`files.upload`

`양돈과_초음파`의 내용물을 압축해서 올려주세요.
(불러온 파일은 세션이 종료되면 사라집니다.)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

업로드 확인

In [ ]:
!ls # 출력: sample_data  양돈과_초음파.zip

압축 풀기

In [ ]:
!unzip -qq '양돈과_초음파.zip' -d ./dataset

In [ ]:
!ls # 출력: dataset  sample_data  양돈과_초음파.zip

In [6]:
!ls dataset/ # 출력: 19_1  21_1  23_1  25_1

19_1  21_1  23_1  25_1


## 02. 데이터 정제

레이블 파일이 존재하지 않거나, 파일은 있지만 비어있는 경우를 제외합니다.

In [ ]:
import os
import glob

#  1) glob으로 모든 BMP 파일 찾기
# 예시 패턴: '/content/양돈과_초음파/*/*.bmp'
pattern = os.path.join('./dataset', '*', '*.bmp')
all_bmp_paths = glob.glob(pattern)

print('총 이미지 수: ', len(all_bmp_paths))

#  2) 레이블(.txt) 파일이 있는 경우만 남기기
img_files = []  # 정제된 이미지 경로를 담는 리스트
for bmp_path in all_bmp_paths:
    # 이미지 파일 경로에서 확장자만 .txt로 바꿔서 레이블 경로 생성
    txt_path = bmp_path[:-4] + '.txt'
    # 레이블 파일이 존재할 때만
    if os.path.exists(txt_path):
        # 레이블 파일이 비어있다면 알리고 넘어감
        if os.path.getsize(txt_path) == 0:
            print(f"{txt_path} is None")
            continue
        img_files.append(bmp_path)
# img_files에는 .bmp/.txt 쌍만 남김

총 이미지 수:  189
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0012.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0022.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0029.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0011.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0017.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0036.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0021.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0033.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0014.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0040.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0019.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0020.txt is None
./dataset/21_1/001_W5741__2021-10-08_14_45_07_frame0041.txt is None


In [8]:
# 정제한 데이터 수 확인
len(img_files)

119

## 03. train/val 스플릿

일정 비율(9:1)로 학습 데이터셋과 검증 데이터셋으로 나눠줍니다.

데이터 수(이미지 및 오브젝트)가 매우 적기 때문에 test 셋은 제외합니다.

테스트 셋까지 포함할 경우에는 8:1:1 비율을 추천합니다.

(더 안정적인 평가를 위해서는 k-fold도 추천합니다.)


`scikit-learn` 라이브러리의 `train_test_split`을 이용합니다.

```
train_test_split(데이터셋 리스트, test_size=테스트셋 비율, shuffle=셔플여부)
```

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_val = train_test_split(img_files, test_size=0.1, shuffle=True, random_state=2000) # random_state는 랜덤 시드를 고정해주는 옵션으로, 제외해도 됩니다.

In [10]:
len(x_train), len(x_val)

(107, 12)

## 04. 구축 및 저장

데이터셋을 YOLO 구조에 맞게 저장합니다.

In [11]:
# 1) 저장 경로 설정
train_img_dir = './PigDataset/train/images'
train_label_dir = './PigDataset/train/labels'
val_img_dir = './PigDataset/val/images'
val_label_dir = './PigDataset/val/labels'

In [12]:
# 2) 디렉토리 생성
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

In [13]:
# 3) 파일 복사 - shutil 라이브러리 이용
import shutil

In [14]:
x_train[0]

'./dataset/23_1/001_W5741__2021-10-10_09_19_35_frame0008.bmp'

In [15]:
# train set
for img_path in x_train:
    shutil.copy(img_path, train_img_dir)   # image
    txt_path = img_path[:-4] + '.txt'
    shutil.copy(txt_path, train_label_dir) # label

In [16]:
# validation set
for img_path in x_val:
    shutil.copy(img_path, val_img_dir)   # image
    txt_path = img_path[:-4] + '.txt'
    shutil.copy(txt_path, val_label_dir) # label

In [17]:
# 확인
!ls PigDataset/

train  val


옵션) 리눅스 `tree` 패키지를 이용해 생성한 데이터셋 디렉토리 구조를 확인합니다.

```
PigDataset/
├── train
│   ├── images
│   └── labels
└── val
    ├── images
    └── labels
2
```

In [18]:
# 설치
!apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (96.6 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [19]:
!tree PigDataset/ -d 2

PigDataset/
├── train
│   ├── images
│   └── labels
└── val
    ├── images
    └── labels
2


In [20]:
!tree PigDataset/train/

PigDataset/train/
├── images
│   ├── 001_W5741__2021-10-08_14_45_07_frame0000.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0001.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0002.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0003.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0004.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0005.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0006.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0007.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0009.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0010.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0013.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0015.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0016.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0018.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0023.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0024.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0025.bmp
│   ├── 001_W5741__2021-10-08_14_45_07_frame0026.bmp
│   ├── 001_W5741

In [21]:
!tree PigDataset/val/

PigDataset/val/
├── images
│   ├── 001_W5741__2021-10-08_14_45_07_frame0008.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0000.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0018.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0024.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0026.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0031.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0033.bmp
│   ├── 001_W5741__2021-10-10_09_19_35_frame0036.bmp
│   ├── 001_W5741__2021-10-12_10_01_06_frame0011.bmp
│   ├── 001_W5741__2021-10-12_10_01_06_frame0026.bmp
│   ├── 001_W5741__2021-10-12_10_01_06_frame0029.bmp
│   └── 21-2_frame21.bmp
└── labels
    ├── 001_W5741__2021-10-08_14_45_07_frame0008.txt
    ├── 001_W5741__2021-10-10_09_19_35_frame0000.txt
    ├── 001_W5741__2021-10-10_09_19_35_frame0018.txt
    ├── 001_W5741__2021-10-10_09_19_35_frame0024.txt
    ├── 001_W5741__2021-10-10_09_19_35_frame0026.txt
    ├── 001_W5741__2021-10-10_09_19_35_frame0031.txt
    ├── 001_W5741__2021-10-10_09_19_

## 05. YAML 파일

데이터셋에 대해 **YOLO에서 사용하는 설정 정보(Config) 파일**인 `yaml`파일을 작성합니다.

YOLO 데이터셋 설정 파일에서는 데이터셋의 경로와 클래스 정보 등을 저장합니다.

작성 및 저장

In [27]:
yaml_content = """
# Train/val/test sets
path:  PigDataset        # 데이터셋 루트 경로
train: train/images     # 학습 이미지 경로 ('path'의 상대 경로)
val:   val/images       # 검증 이미지 경로 ('path'의 상대 경로)
test:                   # 테스트 이미지 경로 (optional)

# Classes
names:
  0: pig
"""

with open('PigDataset/pig.yaml', 'w') as f:
    f.write(yaml_content)

확인

In [28]:
!cat PigDataset/pig.yaml


# Train/val/test sets
path:  PigDataset        # 데이터셋 루트 경로
train: train/images     # 학습 이미지 경로 ('path'의 상대 경로)
val:   val/images       # 검증 이미지 경로 ('path'의 상대 경로)
test:                   # 테스트 이미지 경로 (optional)

# Classes
names:
  0: pig


## 06. 다운로드

구축한 데이터셋을 다운로드합니다.**굵은 텍스트**

압축

In [29]:
!zip PigDataset.zip -r PigDataset

updating: PigDataset/ (stored 0%)
updating: PigDataset/data.yaml (deflated 37%)
updating: PigDataset/val/ (stored 0%)
updating: PigDataset/val/labels/ (stored 0%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0018.txt (deflated 26%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0036.txt (deflated 39%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0024.txt (deflated 29%)
updating: PigDataset/val/labels/001_W5741__2021-10-08_14_45_07_frame0008.txt (deflated 38%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0000.txt (deflated 43%)
updating: PigDataset/val/labels/21-2_frame21.txt (deflated 13%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0033.txt (deflated 38%)
updating: PigDataset/val/labels/001_W5741__2021-10-12_10_01_06_frame0026.txt (deflated 39%)
updating: PigDataset/val/labels/001_W5741__2021-10-10_09_19_35_frame0031.txt (deflated 28%)
updating: PigDataset/val/labels/001_W

In [ ]:
!ls # 출력: dataset  PigDataset  PigDataset.zip  sample_data  양돈과_초음파.zip

다운로드

다운로드 후 로컬 디스크에 저장된 파일을 확인하세요.

In [31]:
files.download('PigDataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>